In [1]:
import boto3 
from datetime import datetime, timezone

In [2]:
client_quota = boto3.client("service-quotas")

In [4]:
quotas = client_quota.list_service_quotas(ServiceCode='sagemaker')['Quotas']

In [5]:
quotas

[{'ServiceCode': 'sagemaker',
  'ServiceName': 'Amazon SageMaker',
  'QuotaArn': 'arn:aws:servicequotas:us-west-2:789636077263:sagemaker/L-E6D074A3',
  'QuotaCode': 'L-E6D074A3',
  'QuotaName': 'Studio CodeEditor Apps running on ml.r6id.large instances',
  'Value': 11.0,
  'Unit': 'None',
  'Adjustable': True,
  'GlobalQuota': False,
  'UsageMetric': {'MetricNamespace': 'AWS/Usage',
   'MetricName': 'ResourceCount',
   'MetricDimensions': {'Class': 'None',
    'Resource': 'studio/CodeEditor-ml.r6id.large',
    'Service': 'SageMaker',
    'Type': 'Resource'},
   'MetricStatisticRecommendation': 'Maximum'},
  'Period': {'PeriodValue': 1, 'PeriodUnit': 'HOUR'},
  'QuotaAppliedAtLevel': 'ACCOUNT'},
 {'ServiceCode': 'sagemaker',
  'ServiceName': 'Amazon SageMaker',
  'QuotaArn': 'arn:aws:servicequotas:us-west-2:789636077263:sagemaker/L-1EC048AB',
  'QuotaCode': 'L-1EC048AB',
  'QuotaName': 'Studio CodeEditor Apps running on ml.r5.4xlarge instances',
  'Value': 4.0,
  'Unit': 'None',
  'Adju

In [33]:
def get_service_quota_code(service,quota_code):
    quotas = client_quota.list_service_quotas(
        ServiceCode=service,
        MaxResults=200
    )['Quotas']
    return {
        quota['QuotaName']:quota['QuotaCode']
        for quota in quotas
    }
    


In [34]:
get_service_quota_code('sagemaker',"")

ClientError: An error occurred (ValidationException) when calling the ListServiceQuotas operation: 1 validation error detected: Value at 'maxResults' failed to satisfy constraint: Member must have value less than or equal to 100

In [6]:
response = client_quota.get_service_quota(ServiceCode='sagemaker',QuotaCode="L-9614C779")

In [7]:
cloudwatch = boto3.client('cloudwatch') 

In [8]:
usage_metric = response['Quota']['UsageMetric']
usage_metric

{'MetricNamespace': 'AWS/Usage',
 'MetricName': 'ResourceCount',
 'MetricDimensions': {'Class': 'None',
  'Resource': 'endpoint/ml.g5.2xlarge',
  'Service': 'SageMaker',
  'Type': 'Resource'},
 'MetricStatisticRecommendation': 'Maximum'}

In [10]:
from datetime import timedelta
now = datetime.now()
past = now - timedelta(days=30)

dimensions = [{"Name":k,"Value":v} for k,v in usage_metric['MetricDimensions'].items()]
metrics = cloudwatch.get_metric_statistics(
    Namespace=usage_metric['MetricNamespace'],
    MetricName=usage_metric['MetricName'],
    Dimensions=dimensions,
    StartTime=past,
    EndTime=now,
    Period=86400,
    # Statistics=['Sum']
)
metrics 

InvalidParameterCombinationException: An error occurred (InvalidParameterCombination) when calling the GetMetricStatistics operation: At least one of the parameters Statistics and ExtendedStatistics must be specified.

In [17]:
metrics 

{'Label': 'ResourceCount',
 'Datapoints': [{'Timestamp': datetime.datetime(2024, 12, 10, 9, 49, tzinfo=tzlocal()),
   'Sum': 66.0,
   'Unit': 'None'},
  {'Timestamp': datetime.datetime(2024, 12, 13, 9, 49, tzinfo=tzlocal()),
   'Sum': 3.0,
   'Unit': 'None'},
  {'Timestamp': datetime.datetime(2024, 12, 16, 9, 49, tzinfo=tzlocal()),
   'Sum': 49.0,
   'Unit': 'None'},
  {'Timestamp': datetime.datetime(2024, 12, 15, 9, 49, tzinfo=tzlocal()),
   'Sum': 36.0,
   'Unit': 'None'},
  {'Timestamp': datetime.datetime(2024, 12, 12, 9, 49, tzinfo=tzlocal()),
   'Sum': 2.0,
   'Unit': 'None'},
  {'Timestamp': datetime.datetime(2024, 12, 11, 9, 49, tzinfo=tzlocal()),
   'Sum': 40.0,
   'Unit': 'None'}],
 'ResponseMetadata': {'RequestId': '134fcc68-e368-45aa-b7cd-5ea2cf9a9597',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '134fcc68-e368-45aa-b7cd-5ea2cf9a9597',
   'content-type': 'text/xml',
   'content-length': '1149',
   'date': 'Tue, 17 Dec 2024 09:49:53 GMT'},
  'RetryAttempts'

In [1]:
s = "{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = \"26 Jul 2024\" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0]['role'] == 'system' %}\n    {%- set system_message = messages[0]['content']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = \"\" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- \"<|start_header_id|>system<|end_header_id|>\\n\\n\" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- \"Environment: ipython\\n\" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- \"Tools: \" + builtin_tools | reject('equalto', 'code_interpreter') | join(\", \") + \"\\n\\n\"}}\n{%- endif %}\n{{- \"Cutting Knowledge Date: December 2023\\n\" }}\n{{- \"Today Date: \" + date_string + \"\\n\\n\" }}\n{%- if tools is not none and not tools_in_user_message %}\n    {{- \"You have access to the following functions. To call a function, please respond with JSON for a function call.\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n{%- endif %}\n{{- system_message }}\n{{- \"<|eot_id|>\" }}\n\n{#- Custom tools are passed in a user message with some extra guidance #}\n{%- if tools_in_user_message and not tools is none %}\n    {#- Extract the first user message so we can plug it in here #}\n    {%- if messages | length != 0 %}\n        {%- set first_user_message = messages[0]['content']|trim %}\n        {%- set messages = messages[1:] %}\n    {%- else %}\n        {{- raise_exception(\"Cannot put tools in the first user message when there's no first user message!\") }}\n{%- endif %}\n    {{- '<|start_header_id|>user<|end_header_id|>\\n\\n' -}}\n    {{- \"Given the following functions, please respond with a JSON for a function call \" }}\n    {{- \"with its proper arguments that best answers the given prompt.\\n\\n\" }}\n    {{- 'Respond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}.' }}\n    {{- \"Do not use variables.\\n\\n\" }}\n    {%- for t in tools %}\n        {{- t | tojson(indent=4) }}\n        {{- \"\\n\\n\" }}\n    {%- endfor %}\n    {{- first_user_message + \"<|eot_id|>\"}}\n{%- endif %}\n\n{%- for message in messages %}\n    {%- if not (message.role == 'ipython' or message.role == 'tool' or 'tool_calls' in message) %}\n        {{- '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n'+ message['content'] | trim + '<|eot_id|>' }}\n    {%- elif 'tool_calls' in message %}\n        {%- if not message.tool_calls|length == 1 %}\n            {{- raise_exception(\"This model only supports single tool-calls at once!\") }}\n        {%- endif %}\n        {%- set tool_call = message.tool_calls[0].function %}\n        {%- if builtin_tools is defined and tool_call.name in builtin_tools %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- \"<|python_tag|>\" + tool_call.name + \".call(\" }}\n            {%- for arg_name, arg_val in tool_call.arguments | items %}\n                {{- arg_name + '=\"' + arg_val + '\"' }}\n                {%- if not loop.last %}\n                    {{- \", \" }}\n                {%- endif %}\n                {%- endfor %}\n            {{- \")\" }}\n        {%- else  %}\n            {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' -}}\n            {{- '{\"name\": \"' + tool_call.name + '\", ' }}\n            {{- '\"parameters\": ' }}\n            {{- tool_call.arguments | tojson }}\n            {{- \"}\" }}\n        {%- endif %}\n        {%- if builtin_tools is defined %}\n            {#- This means we're in ipython mode #}\n            {{- \"<|eom_id|>\" }}\n        {%- else %}\n            {{- \"<|eot_id|>\" }}\n        {%- endif %}\n    {%- elif message.role == \"tool\" or message.role == \"ipython\" %}\n        {{- \"<|start_header_id|>ipython<|end_header_id|>\\n\\n\" }}\n        {%- if message.content is mapping or message.content is iterable %}\n            {{- message.content | tojson }}\n        {%- else %}\n            {{- message.content }}\n        {%- endif %}\n        {{- \"<|eot_id|>\" }}\n    {%- endif %}\n{%- endfor %}\n{%- if add_generation_prompt %}\n    {{- '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}\n{%- endif %}\n"

In [2]:
print(s)

{{- bos_token }}
{%- if custom_tools is defined %}
    {%- set tools = custom_tools %}
{%- endif %}
{%- if not tools_in_user_message is defined %}
    {%- set tools_in_user_message = true %}
{%- endif %}
{%- if not date_string is defined %}
    {%- set date_string = "26 Jul 2024" %}
{%- endif %}
{%- if not tools is defined %}
    {%- set tools = none %}
{%- endif %}

{#- This block extracts the system message, so we can slot it into the right place. #}
{%- if messages[0]['role'] == 'system' %}
    {%- set system_message = messages[0]['content']|trim %}
    {%- set messages = messages[1:] %}
{%- else %}
    {%- set system_message = "" %}
{%- endif %}

{#- System message + builtin tools #}
{{- "<|start_header_id|>system<|end_header_id|>\n\n" }}
{%- if builtin_tools is defined or tools is not none %}
    {{- "Environment: ipython\n" }}
{%- endif %}
{%- if builtin_tools is defined %}
    {{- "Tools: " + builtin_tools | reject('equalto', 'code_interpreter') | join(", ") + "\n\n"}}
{%- endif